# Code 6
- Preprocessing Improvement
- Decision Tree

## 1/ Import Libraries

In [1]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [2]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [4]:
#CodeSection3
train = pd.read_csv("/mntDrive/MyDrive/__Data_Science/3 HR Analytics/input/train.csv")
test = pd.read_csv("/mntDrive/MyDrive/__Data_Science/3 HR Analytics/input/test.csv")

## 3/ Combine both Datasets (train, test)

In [5]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [6]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

enrollee_id                   0
city                          0
city_development_index        0
gender                     7486
relevent_experience           0
enrolled_university         621
education_level             852
major_discipline           5231
experience                  103
company_size               8830
company_type               9369
last_new_job                671
training_hours                0
target                    15021
dtype: int64

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [7]:
#CodeSection6
# Identify all Numerical and Categorical features
numerical = ['city_development_index', 'training_hours']
categorical = ['gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type', 'last_new_job']

### 4.2/ Outlier Strategy

In [8]:
#CodeSection7

# Write a loop to do the same
for num_var in numerical:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [9]:
#CodeSection8

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Decision tree
from sklearn.tree import DecisionTreeClassifier

### 4.4/ Build Pipeline

In [10]:
#CodeSection9
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    
    (make_pipeline(SimpleImputer(strategy = 'median'),
                   StandardScaler()),numerical),


    (make_pipeline(SimpleImputer(strategy='most_frequent'),
                                 OrdinalEncoder()),categorical)
    
    )        

### 4.5/ Divide Data into X and y

In [11]:
#CodeSection10
X = train.drop(['target','enrollee_id'], axis =  1)
y = train['target']

## 5/ Build Model and Fit

In [22]:
#CodeSection11
# Create Model Pipeline and Initiate Model
model = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth = 5, class_weight = 'balanced'))

In [23]:
#CodeSection12 
# Fit Model
model.fit(X,y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## 6/ Check Accuracy of Model on Train Data

In [24]:
#CodeSection13
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [26]:
#CodeSection14
# Import metrics library
from sklearn.metrics import accuracy_score

In [27]:
#CodeSection15
# get Actual "y" variables (use "y_true" as variable)
y_true = y

In [28]:
#CodeSection16
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

 Train Accuracy : 69.8%


## 7/ Predict and Submission

### Predict on "Test Data"

In [29]:
#CodeSection17
# Get all the X Variables from the Test Dataset
X_test = test.drop(['enrollee_id'], axis =  1)

In [30]:
#CodeSection18
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [31]:
#CodeSection19
submission = pd.DataFrame({
    'enrollee_id' : test['enrollee_id'],
    'target' : X_test_prep
})

### Export Submission File

In [32]:
#CodeSection20
submission.to_csv('/mntDrive/MyDrive/__Data_Science/3 HR Analytics/output/O6_DT_Pipeline.csv', index = False)

In [ ]:
# LB Accuracy : 0.6805